A következő jegyzetfüzetek futtatásához, ha még nem tetted meg, be kell állítanod az openai kulcsot a .env fájlban `OPENAI_API_KEY` néven.


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Ezután betöltjük az Embedding Indexet egy Pandas Dataframe-be. Az Embedding Index egy JSON fájlban van tárolva, amelynek neve `embedding_index_3m.json`. Az Embedding Index tartalmazza az egyes YouTube átiratok beágyazásait 2023 októberének végéig.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Ezután létrehozunk egy `get_videos` nevű függvényt, amely az Embedding Indexben keres a lekérdezés alapján. A függvény visszaadja az 5 leginkább a lekérdezéshez hasonló videót. A függvény a következőképpen működik:

1. Először létrejön egy másolat az Embedding Indexről.
2. Ezután az OpenAI Embedding API segítségével kiszámítjuk a lekérdezés Embeddingjét.
3. Majd létrehozunk egy új oszlopot az Embedding Indexben `similarity` néven. A `similarity` oszlop a lekérdezés Embeddingje és az egyes videószegmensek Embeddingje közötti koszinusz hasonlóságot tartalmazza.
4. Ezután az Embedding Indexet a `similarity` oszlop alapján szűrjük. Csak azok a videók maradnak benne, amelyek koszinusz hasonlósága nagyobb vagy egyenlő 0,75-tel.
5. Végül az Embedding Indexet a `similarity` oszlop szerint rendezzük, és visszaadjuk az 5 legjobb videót.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Ez a függvény nagyon egyszerű, csak kiírja a keresési lekérdezés eredményeit.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Először az Embedding Index betöltődik egy Pandas Dataframe-be.
2. Ezután a felhasználót lekérdezik, hogy adjon meg egy lekérdezést.
3. Majd meghívódik a `get_videos` függvény, hogy megkeresse a lekérdezést az Embedding Indexben.
4. Végül meghívódik a `display_results` függvény, hogy megjelenítse az eredményeket a felhasználónak.
5. Ezután a felhasználót ismét lekérdezik egy újabb lekérdezés megadására. Ez a folyamat addig folytatódik, amíg a felhasználó be nem írja az `exit` parancsot.

![](../../../../translated_images/hu/notebook-search.1e320b9c7fcbb0bc.webp)

A rendszer lekérdezi, hogy adjon meg egy lekérdezést. Írjon be egy lekérdezést, majd nyomja meg az Entert. Az alkalmazás visszaad egy listát a lekérdezéshez releváns videókról. Az alkalmazás egy linket is visszaad a videó azon helyére, ahol a kérdésre adott válasz található.

Íme néhány kipróbálható lekérdezés:

- Mi az Azure Machine Learning?
- Hogyan működnek a konvolúciós neurális hálózatok?
- Mi az a neurális hálózat?
- Használhatom a Jupyter Notebookokat az Azure Machine Learninggel?
- Mi az az ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Jogi nyilatkozat**:
Ezt a dokumentumot az AI fordító szolgáltatás, a [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével fordítottuk le. Bár a pontosságra törekszünk, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az anyanyelvén tekintendő hiteles forrásnak. Fontos információk esetén professzionális emberi fordítást javaslunk. Nem vállalunk felelősséget a fordítás használatából eredő félreértésekért vagy félreértelmezésekért.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
